In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf

# Load Dataset
# Replace this with your dataset if needed
uploaded_nutrition_data = pd.read_csv('/content/nurtition_data_recom.csv')

print(uploaded_nutrition_data.head())

         class  Calories (kcal)  Protein (g)  Carbohydrates (g)  Fat (g)  \
0   Ayam Bakar              167        25.01               0.00     6.63   
1  Ayam Geprek              263        17.61               7.60    17.99   
2  Ayam Goreng              260        21.93              10.76    14.55   
3        Bakso              180        12.00              15.00     8.00   
4   Ikan Bakar              200        30.00               0.00     8.00   

   Fiber (g)  
0        0.0  
1        0.8  
2        1.4  
3        1.0  
4        0.0  


In [8]:
# Simulate BMI and Goal data
np.random.seed(42)
uploaded_nutrition_data['BMI'] = np.random.uniform(18, 35, len(uploaded_nutrition_data))
uploaded_nutrition_data['goal'] = np.random.choice(['weight_loss', 'weight_gain', 'maintenance'], len(uploaded_nutrition_data))

# Function to Assign Labels
def assign_label(row, goal):
    if goal == 'weight_loss':
        return 1 if row['Calories (kcal)'] < 300 and row['Protein (g)'] > 5 and row['Fiber (g)'] > 0.5 else 0
    elif goal == 'weight_gain':
        return 1 if row['Calories (kcal)'] > 250 and row['Protein (g)'] > 15 and row['Fat (g)'] > 10 else 0
    elif goal == 'maintenance':
        return 1 if (150 <= row['Calories (kcal)'] <= 300 and
                     row['Protein (g)'] > 8 and
                     row['Carbohydrates (g)'] > 5 and
                     5 <= row['Fat (g)'] <= 17) else 0
    return 0

# Generate Labels
uploaded_nutrition_data['weight_loss_label'] = uploaded_nutrition_data.apply(assign_label, axis=1, goal='weight_loss')
uploaded_nutrition_data['weight_gain_label'] = uploaded_nutrition_data.apply(assign_label, axis=1, goal='weight_gain')
uploaded_nutrition_data['maintenance_label'] = uploaded_nutrition_data.apply(assign_label, axis=1, goal='maintenance')

# Menampilkan beberapa data setelah penambahan kategori diet
print(uploaded_nutrition_data[['class', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)', 'weight_loss_label', 'weight_gain_label', 'maintenance_label']].head())

         class  Calories (kcal)  Protein (g)  Carbohydrates (g)  Fat (g)  \
0   Ayam Bakar              167        25.01               0.00     6.63   
1  Ayam Geprek              263        17.61               7.60    17.99   
2  Ayam Goreng              260        21.93              10.76    14.55   
3        Bakso              180        12.00              15.00     8.00   
4   Ikan Bakar              200        30.00               0.00     8.00   

   Fiber (g)  weight_loss_label  weight_gain_label  maintenance_label  
0        0.0                  0                  0                  0  
1        0.8                  1                  1                  0  
2        1.4                  1                  1                  1  
3        1.0                  1                  0                  1  
4        0.0                  0                  0                  0  


In [9]:
# Prepare Features and Labels (example: weight_loss)
X = uploaded_nutrition_data[['BMI', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)']]
y = uploaded_nutrition_data['weight_loss_label']

# One-hot encode the 'goal' feature
goal_encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
goal_encoded = goal_encoder.fit_transform(uploaded_nutrition_data[['goal']])
goal_columns = goal_encoder.get_feature_names_out(['goal'])
goal_df = pd.DataFrame(goal_encoded, columns=goal_columns)

# Combine encoded goal with features
X = pd.concat([X.reset_index(drop=True), goal_df.reset_index(drop=True)], axis=1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the Neural Network Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=4, validation_split=0.2, verbose=1)

# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Display the Test Accuracy
print(f"Test Accuracy: {accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 306ms/step - accuracy: 0.4976 - loss: 0.6783 - val_accuracy: 0.5714 - val_loss: 0.6584
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5429 - loss: 0.6594 - val_accuracy: 0.7143 - val_loss: 0.6399
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5619 - loss: 0.6269 - val_accuracy: 0.7143 - val_loss: 0.6251
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5244 - loss: 0.6498 - val_accuracy: 1.0000 - val_loss: 0.6076
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8536 - loss: 0.6042 - val_accuracy: 1.0000 - val_loss: 0.5954
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9131 - loss: 0.5322 - val_accuracy: 1.0000 - val_loss: 0.5791
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8762 - loss: 0.5721 - val_accuracy: 1.0000 - val_loss: 0.5622
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8673 - loss: 0.5236 - val_accuracy: 0.8571 - val_loss: 0.5441
Ep

## Inference

In [ ]:
# Input baru tanpa memperhatikan urutan
user_input = {
    'BMI': 25.0,
    'Calories (kcal)': 250,
    'Protein (g)': 10,
    'Carbohydrates (g)': 30,
    'Fat (g)': 10,
    'Fiber (g)': 4,
    'goal_weight_loss': 0,
    'goal_weight_gain': 0,
    'goal_maintenance': 1
}

# Konversi ke DataFrame
new_data = pd.DataFrame([user_input])

# Lengkapi kolom yang hilang
for col in X.columns:
    if col not in new_data.columns:
        new_data[col] = 0  # Default value for missing columns

# Pastikan urutan sesuai dengan data training
new_data = new_data[X.columns]

# Scale input data
new_data_scaled = scaler.transform(new_data)

# Predict using the model
prediction = model.predict(new_data_scaled)
predicted_label = (prediction > 0.5).astype(int)

print(f"Predicted Label: {predicted_label[0][0]} (1 = cocok, 0 = tidak cocok)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicted Label: 1 (1 = cocok, 0 = tidak cocok)


In [ ]:
# Prepare Features for display based on labels
weight_loss_recommendations = uploaded_nutrition_data[uploaded_nutrition_data['weight_loss_label'] == 1]
weight_gain_recommendations = uploaded_nutrition_data[uploaded_nutrition_data['weight_gain_label'] == 1]
maintenance_recommendations = uploaded_nutrition_data[uploaded_nutrition_data['maintenance_label'] == 1]

# Menampilkan kolom yang relevan (misalnya, class, kalori, protein, dll)
weight_loss_display = weight_loss_recommendations[['class', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)']]
weight_gain_display = weight_gain_recommendations[['class', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)']]
maintenance_display = maintenance_recommendations[['class', 'Calories (kcal)', 'Protein (g)', 'Carbohydrates (g)', 'Fat (g)', 'Fiber (g)']]

# Fungsi untuk mendapatkan rekomendasi yang sudah diurutkan
def get_sorted_recommendations(goal, sort_by='Calories (kcal)', ascending=True):
    if goal == 'weight_loss':
        recommendations = weight_loss_display
        # Sort: Kalori terendah, Protein tinggi, Serat tinggi
        recommendations_sorted = recommendations.sort_values(by=['Calories (kcal)', 'Protein (g)', 'Fiber (g)'], ascending=[True, False, False])

    elif goal == 'weight_gain':
        recommendations = weight_gain_display
        # Sort: Kalori tertinggi, Protein tinggi, Lemak tinggi
        recommendations_sorted = recommendations.sort_values(by=['Calories (kcal)', 'Protein (g)', 'Fat (g)'], ascending=[False, False, False])

    elif goal == 'maintenance':
        recommendations = maintenance_display
        # Sort: Kalori moderat, Protein cukup, Lemak moderat
        recommendations_sorted = recommendations.sort_values(by=['Calories (kcal)', 'Protein (g)', 'Fat (g)'], ascending=[True, False, True])
    else:
        print("Invalid goal!")
        return None

    return recommendations_sorted

# Contoh penggunaan fungsi untuk rekomendasi weight loss yang diurutkan berdasarkan kalori
goal = 'weight_loss'  # Bisa diganti 'weight_gain' atau 'maintenance'
sorted_recommendations = get_sorted_recommendations(goal)
print(f"\nSorted {goal.capitalize()} Recommendations:")
print(sorted_recommendations)

# Misalnya Anda ingin rekomendasi weight gain, urutkan berdasarkan Protein
goal = 'weight_gain'
sorted_recommendations = get_sorted_recommendations(goal)
print(f"\nSorted {goal.capitalize()} Recommendations (by Protein):")
print(sorted_recommendations)

# Misalnya Anda ingin rekomendasi maintenance, urutkan berdasarkan Fat
goal = 'maintenance'
sorted_recommendations = get_sorted_recommendations(goal)
print(f"\nSorted {goal.capitalize()} Recommendations (by Fat):")
print(sorted_recommendations)


Sorted Weight_loss Recommendations:
                         class  Calories (kcal)  Protein (g)  \
22                  Sayur Asem              150         6.00   
32              Tumis Kangkung              150         6.00   
31  Bubur Ayam (tanpa kerupuk)              180        20.00   
25           Oseng Tempe Kecap              180        12.00   
3                        Bakso              180        12.00   
28            Gulai Tahu Tempe              200        12.00   
26                 Sayur Lodeh              210         8.00   
30               Ayam Taliwang              220        30.00   
11                   Soto Ayam              220        20.00   
16                   Gado-Gado              220        10.00   
10                   Sate Ayam              240        25.00   
19                      Pempek              240        14.00   
24                  Pecel Lele              250        25.00   
8                        Pecel              250        10.00   
6  

In [ ]:
model.save("/content/model2.h5")

In [ ]:
!tensorflowjs_converter --input_format=keras /content/model2.h5 /content/tfjs_model

2024-12-01 15:59:12.938313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733068752.985356    4162 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733068752.998976    4162 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file


In [ ]:
!zip -r model2_tfjs.zip /content/tfjs_model

  adding: content/tfjs_model/ (stored 0%)
  adding: content/tfjs_model/group1-shard1of1.bin (deflated 8%)
  adding: content/tfjs_model/model.json (deflated 81%)
